In [ ]:
sos run xqtl-pipeline/code/misc/build_container.ipynb singularity_generator\
    --container_list xqtl-pipeline/code/misc/xQTL_conda_packages1.csv \
    --env bioinfo fastenloc leafcutter METAL methylation polyfun psichomics rna_quantification stephenslab TensorQTL

In [ ]:
sos run xqtl-pipeline/code/misc/build_container.ipynb docker_generator\
    --container_list xqtl-pipeline/code/misc/xQTL_conda_packages1.csv \
    --env bioinfo fastenloc leafcutter METAL methylation polyfun psichomics rna_quantification stephenslab TensorQTL

In [ ]:
[global]
parameter: container_list = path
parameter: env = list
parameter: output = env
parameter: cwd=path("output8")

In [ ]:
[docker_generator_1, singularity_generator_1]
input: container_list, for_each="env"
output: f'{cwd}/{_env}/{_env}.yml'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    import os
    import yaml
    import pandas as pd

    def get_package_and_version(row):
        package_name = row['Conda package'].split("/")[-1]
        version = row['CondaVersion'] if pd.notnull(row['CondaVersion']) and row['CondaVersion'] != '' else row['Version']
        return package_name, f"{package_name}={version}" if pd.notna(version) and version != 'nan' else package_name

    container_list = pd.read_csv("$[_input]")
    container_tempt = dict()

    container_list = container_list[container_list.Channel.notna()]
    environment_name = "$[_env]"
    container_tempt["name"] = environment_name
    container_tempt["channels"] = list(set(container_list.query("Envir                         onment.str.contains(@environment_name) | Environment == 'Universial'").Channel))

    subset_list = container_list[(container_list.Environment.str.contains(environment_name)) & (container_list.Channel.isin( container_tempt['channels']))]
    dependencies_dict = {package_name: package_version for package_name, package_version in [get_package_and_version(row) for index, row in subset_list.iterrows()]}
    

    subset_list2 = container_list[(container_list.Environment == 'Universial') & (container_list.Channel.isin(container_tempt['channels']))]
    universal_dependencies = [get_package_and_version(row)[1] for index, row in subset_list2.iterrows() if get_package_and_version(row)[0] not in dependencies_dict]

    container_tempt["dependencies"] = list(dependencies_dict.values()) + universal_dependencies

    container_tempt["channels"].sort(reverse=True)
    container_tempt["channels"].append("nodefaults")
    container_tempt["channels"].sort(reverse=True)

    class IndentDumper(yaml.Dumper):
        def increase_indent(self, flow=False, indentless=False):
            return super(IndentDumper, self).increase_indent(flow, False)
    with open("$[_output]", 'w') as f:
        yaml.dump(container_tempt,f,sort_keys = False, Dumper=IndentDumper )

In [ ]:
[singularity_generator_2]
output: f'{_input:na}.sif'
report: output=f'{_input:da}/build.def'
    Bootstrap: docker
    From: mambaorg/micromamba
    %setup
        cp *.yml ${SINGULARITY_ROOTFS}
    %post
        micromamba env create --quiet --yes --file *.yml
        micromamba clean --yes
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    cd $[_input:d]
    rm -f $[_output]
    singularity build --fakeroot $[_output] build.def
    md5sum $[_output] > $[_output].md5

In [ ]:
[docker_generator_2]
output: f'{_input:na}.tar'
report: output=f'{_input:da}/Dockerfile', expand=True
    FROM mambaorg/micromamba

    COPY {_input:nb}.yml ./

    RUN micromamba shell -n base -- bash -c "micromamba env create --quiet --yes --file {_input:nb}.yml" && \
        micromamba clean --yes
report: output=f'{_input:da}/docker-compose.yml', expand=True
    version: '3'
    services:
      {_input:nb}:
        build: {_input:da} # path to directory containing Dockerfile
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    cd $[_input:d]
    docker-compose build
    docker save -o $[_output] $[_input:nb]_$[_input:nb]:latest 